In [1]:
import os
import re
import torch
import wandb
import pandas as pd
from PIL import Image
from pathlib import Path
from tqdm import tqdm
from unsloth import FastVisionModel


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
Unsloth: Your Flash Attention 2 installation seems to be broken?
A possible explanation is you have a new CUDA version which isn't
yet compatible with FA2? Please file a ticket to Unsloth or FA2.
We shall now use Xformers instead, which does not have any performance hits!
We found this negligible impact by benchmarking on 1x A100.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [2]:
# ── CONFIG ───────────────────────────────────────────────────────────────────
MODEL_PATH  = "/teamspace/studios/this_studio/qwen2vl_amazon_lora"
VAL_CSV     = "/teamspace/studios/this_studio/small_val.csv"
IMAGE_DIR   = "/teamspace/studios/this_studio/images/"
OUTPUT_CSV  = "/teamspace/studios/this_studio/val_predictions.csv"
# ─────────────────────────────────────────────────────────────────────────────

In [3]:

ALLOWED_UNITS = {
    "centimetre", "foot", "inch", "metre", "millimetre", "yard",
    "cubic foot", "cubic inch", "cup", "decilitre", "fluid ounce",
    "gallon", "imperial gallon", "litre", "microlitre", "millilitre",
    "pint", "quart", "gram", "kilogram", "microgram", "milligram",
    "ounce", "pound", "ton", "kilovolt", "millivolt", "volt",
    "kilowatt", "watt", "decibel", "centilitre", "cubic centimetre", "tonne",
}

In [4]:
def postprocess(raw: str) -> str:
    raw = raw.strip()
    if raw.upper() in ["NA", "N/A", "NONE", "NOT FOUND", ""]:
        return ""
    pattern = re.compile(r'(-?\d+(?:\.\d+)?)\s+([a-zA-Z\s]+)')
    match = pattern.search(raw)
    if not match:
        return ""
    number = match.group(1)
    unit   = match.group(2).strip().lower()
    if unit.replace('ter', 'tre') in ALLOWED_UNITS:
        unit = unit.replace('ter', 'tre')
    if unit.replace('feet', 'foot') in ALLOWED_UNITS:
        unit = unit.replace('feet', 'foot')
    if unit not in ALLOWED_UNITS:
        return ""
    try:
        return f"{float(number)} {unit}"
    except:
        return ""


In [5]:
def compute_f1(predictions, ground_truths):
    tp, fp, fn = 0, 0, 0
    for pred, gt in zip(predictions, ground_truths):
        pred, gt = str(pred).strip(), str(gt).strip()
        if pred != "" and gt != "" and pred == gt:
            tp += 1
        elif pred != "" and gt != "" and pred != gt:
            fp += 1
        elif pred != "" and gt == "":
            fp += 1
        elif pred == "" and gt != "":
            fn += 1
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
    recall    = tp / (tp + fn) if (tp + fn) > 0 else 0.0
    f1        = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0.0
    return {"f1": f1, "precision": precision, "recall": recall, "tp": tp, "fp": fp, "fn": fn}

In [6]:
def build_prompt(entity_name):
    units_str = ", ".join(sorted(ALLOWED_UNITS))
    return (
        f"What is the {entity_name}? "
        f"Reply with only the value and unit from: {units_str}. "
        f"If not visible, reply with empty string."
    )


In [7]:
# Load model
print("Loading model...")
model, processor = FastVisionModel.from_pretrained(
    MODEL_PATH,
    load_in_4bit=True,
)
FastVisionModel.for_inference(model)
model.eval()
device = next(model.parameters()).device

# Load val data
val_df = pd.read_csv(VAL_CSV)
print(f"Val samples: {len(val_df)}")

predictions, ground_truths = [], []

Loading model...
==((====))==  Unsloth 2026.2.1: Fast Qwen2_Vl patching. Transformers: 5.3.0.dev0.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.278 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.10.0+cu128. CUDA: 8.9. CUDA Toolkit: 12.8. Triton: 3.6.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.35. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading weights:   0%|          | 0/729 [00:00<?, ?it/s]

The image processor of type `Qwen2VLImageProcessor` is now loaded as a fast processor by default, even if the model checkpoint was saved with a slow processor. This is a breaking change and may produce slightly different outputs. To continue using the slow processor, instantiate this class with `use_fast=False`. 


Val samples: 1996


In [8]:

# Verify GPU
print(next(model.parameters()).device)  # must show cuda:0

cuda:0


In [10]:
for _, row in tqdm(val_df.iterrows(), total=len(val_df)):
    image_path = f"{IMAGE_DIR}{Path(row['image_link']).name}"
    gt = str(row["entity_value"]).strip()

    if not os.path.exists(image_path):
        predictions.append("")
        ground_truths.append(gt)
        continue

    try:
        image = Image.open(image_path).convert("RGB")
    except:
        predictions.append("")
        ground_truths.append(gt)
        continue

    messages = [{
        "role": "user",
        "content": [
            {"type": "image"},
            {"type": "text", "text": build_prompt(row["entity_name"])},
        ],
    }]

    text_prompt = processor.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )
    inputs = processor(
        text=[text_prompt], images=[image],
        return_tensors="pt", padding=True
    ).to(device)
    with torch.inference_mode():  # faster than torch.no_grad()
        output_ids = model.generate(
            **inputs,
            max_new_tokens=16,   # ← reduce from 32, entity values are short
            do_sample=False,
            use_cache=True,
        )
    generated = output_ids[0][inputs["input_ids"].shape[1]:]
    raw = processor.decode(generated, skip_special_tokens=True).strip()
    predictions.append(postprocess(raw))
    ground_truths.append(gt)
    

100%|██████████| 1996/1996 [1:20:42<00:00,  2.43s/it]


In [11]:
# Compute F1
metrics = compute_f1(predictions, ground_truths)
print("\n" + "="*50)
print("VAL SET RESULTS")
print("="*50)
print(f"  F1 Score:  {metrics['f1']:.4f}")
print(f"  Precision: {metrics['precision']:.4f}")
print(f"  Recall:    {metrics['recall']:.4f}")
print(f"  TP: {metrics['tp']}  FP: {metrics['fp']}  FN: {metrics['fn']}")

# Per entity breakdown
print("\nPer-entity F1:")
for entity in val_df["entity_name"].unique():
    mask  = val_df["entity_name"] == entity
    idxs  = val_df[mask].index.tolist()
    e_p   = [predictions[i] for i in range(len(val_df)) if val_df.iloc[i].name in idxs]
    e_g   = [ground_truths[i] for i in range(len(val_df)) if val_df.iloc[i].name in idxs]
    e_m   = compute_f1(e_p, e_g)
    print(f"  {entity:30s}  F1: {e_m['f1']:.4f}")

# Save predictions
val_df["prediction"] = predictions
val_df.to_csv(OUTPUT_CSV, index=False)
print(f"\nSaved to {OUTPUT_CSV}")


VAL SET RESULTS
  F1 Score:  0.7387
  Precision: 0.6466
  Recall:    0.8615
  TP: 1169  FP: 639  FN: 188

Per-entity F1:
  depth                           F1: 0.7201
  height                          F1: 0.7950
  item_volume                     F1: 0.7333
  item_weight                     F1: 0.7552
  maximum_weight_recommendation   F1: 0.3571
  voltage                         F1: 0.7500
  wattage                         F1: 0.8846
  width                           F1: 0.6421

Saved to /teamspace/studios/this_studio/val_predictions.csv
